## TESTING NOTEBOOK FOR REPAIRING PDF'S

In [ ]:
from pathlib import Path
import deepdoctection as dd
import spacy
from bs4 import BeautifulSoup


In [199]:
config_overwrite = ["LANGUAGE='nld'","TEXT_ORDERING.INCLUDE_RESIDUAL_TEXT_CONTAINER=True"]
analyzer = dd.get_dd_analyzer(config_overwrite=config_overwrite)

nlp = spacy.load("nl_core_news_sm")


[0222 11:21.49 @dd.py:422]  INF  Config: 
 {'DEVICE': 'cpu',
 'LANGUAGE': 'nld',
 'LAYOUT_NMS_PAIRS': {'COMBINATIONS': None, 'PRIORITY': None, 'THRESHOLDS': None},
 'LIB': 'PT',
 'OCR': {'CONFIG': {'TESSERACT': 'dd/conf_tesseract.yaml'},
         'USE_DOCTR': False,
         'USE_TESSERACT': True,
         'USE_TEXTRACT': False,
         'WEIGHTS': {'DOCTR_RECOGNITION': {'PT': 'doctr/crnn_vgg16_bn/pt/crnn_vgg16_bn-9762b0b0.pt',
                                           'TF': 'doctr/crnn_vgg16_bn/tf/crnn_vgg16_bn-76b7f2c6.zip'},
                     'DOCTR_WORD': {'PT': 'doctr/db_resnet50/pt/db_resnet50-ac60cadc.pt',
                                    'TF': 'doctr/db_resnet50/tf/db_resnet50-adcafc63.zip'}}},
 'PT': {'CELL': {'FILTER': None,
                 'WEIGHTS': 'cell/d2_model_1849999_cell_inf_only.pt',
                 'WEIGHTS_TS': 'cell/d2_model_1849999_cell_inf_only.ts'},
        'ITEM': {'FILTER': None,
                 'PAD': {'BOTTOM': 60, 'LEFT': 60, 'RIGHT': 60, 'TOP': 

[0222 11:21.49 @model.py:970]  ERR  File downloaded from deepdoctection/d2_casc_rcnn_X_32xd4_50_FPN_GN_2FC_publaynet_inference_only does not match the expected size! You may have downloaded a broken file, or the upstream may have modified the file.
[0222 11:21.51 @model.py:970]  ERR  File downloaded from deepdoctection/d2_casc_rcnn_X_32xd4_50_FPN_GN_2FC_pubtabnet_rc_inference_only does not match the expected size! You may have downloaded a broken file, or the upstream may have modified the file.
[0222 11:21.54 @model.py:970]  ERR  File downloaded from deepdoctection/d2_casc_rcnn_X_32xd4_50_FPN_GN_2FC_pubtabnet_c_inference_only does not match the expected size! You may have downloaded a broken file, or the upstream may have modified the file.


In [299]:
FILE = "evaluation/pdfs/tk-beslisnota-bij-kamerbrief-inzake-verdeelbesluiten"
# FILE = "evaluation/pdfs/Adviesaanvraag+inzake+de+civiel-militaire+ontwikkelingen+in+het+ruimtedomein.pdf"

path = Path.cwd() / (FILE + ".pdf")

df = analyzer.analyze(path=path)


In [300]:
doc = iter(df)
# pdf = [page for page in doc]

pdf = [next(doc)]


[0222 13:02.44 @doctectionpipe.py:84]  INF  Processing tk-beslisnota-bij-kamerbrief-inzake-verdeelbesluiten_0.pdf


[0222 13:02.54 @context.py:126]  INF  ImageLayoutService total: 7.412 sec.
[0222 13:02.54 @context.py:126]  INF  SubImageLayoutService total: 0.0001 sec.
[0222 13:02.54 @context.py:126]  INF  SubImageLayoutService total: 0.0 sec.
[0222 13:02.54 @context.py:126]  INF  TableSegmentationService total: 0.0001 sec.
[0222 13:02.54 @context.py:126]  INF  TableSegmentationRefinementService total: 0.0 sec.
[0222 13:02.56 @context.py:126]  INF  TextExtractionService total: 2.4387 sec.
[0222 13:02.56 @context.py:126]  INF  MatchingService total: 0.0047 sec.
[0222 13:02.56 @context.py:126]  INF  TextOrderService total: 0.0284 sec.


In [301]:
for layout in sorted(pdf[0].layouts, key=lambda x: x.reading_order):
    print(f"{layout.reading_order:2d} - {layout.category_name:5s} : {layout.text}")

# sorted(pdf[0].layouts, key=lambda x: x.reading_order)[5].text


 1 - line  : OA
 2 - line  : Tweede Kamer
 3 - title : nota
 4 - text  : Beslisnota Kamerbrief inzake verdeelbesluiten
 5 - title : 1 Aanleiding
 6 - text  : Op 31 oktober 2024 ontving u de verslagen van de commissarissen van de Koning in het kader van de ‘Wet gemeentelijke taak mogelijk maken asielopvang- voorzieningen’ (hierna: de wet). Deze heeft u op 1 november jl. aan de Kamer gezonden. In de verslagen doen de commissarissen verslag van het overleg tussen gemeenten, provincie en het COA over de uitvoering van de taken op grond van de wet en geven ze aan hoe de verdeling van reguliere en amv- opvangplaatsen voor de daaropvolgende twee jaren er uit zal zien. Op grond van artikel 5, eerste lid, van de wet neemt u voor 31 december 2024 een verdeelbesluit per provincie. In deze besluiten worden de opvangplaatsen per provincie, zoals bepaald in de provinciale opvangopgave die voor 1 februari van hetzelfde kalenderjaar bekend is gemaakt, verdeeld over de in het besluit aangewezen gemeent

In [310]:
!pdftotext {FILE+".pdf"} -nopgbrk -l 1

file = open(FILE+".txt")
text = file.read()
file.close()


In [311]:
class Word:
    def __init__(self, token, text, line, index):
        self.token = token
        self.text = text

        self.line = line
        self.index = index

        self.border = 0
        self.missing = 0

    def __repr__(self):
        return f"{self.text} = 'B'{self.border}({self.line}:{self.index}) - missing {self.missing} times."


In [312]:
# PdfToText
ptt = {}
n = 4    # 4-gram

def doc_get_word_1(doc):
    for token in doc:
        if not token.is_punct:
            return token
    return None

tokens = [Word(doc_get_word_1(nlp.make_doc(t)), t, l, i) for l, s in enumerate(text.split("\n")) for i, t in enumerate(s.split(" ")) if t]
tokens = [t for t in tokens if t.token]

ptt['tokens'] = tokens
ptt['ngrams'] = {" ".join(t.token.text for t in tokens[i:i+n]) : tokens[i:i+n] for i in range(len(tokens) - n + 1)}


In [313]:
for i in range(len(ptt['tokens'])):
    if i != (len(ptt['tokens']) - 1) and (ptt['tokens'][i+1].line - ptt['tokens'][i].line) > 1:
        ptt['tokens'][i].border = 1
    if i != 0 and (ptt['tokens'][i].line - ptt['tokens'][i-1].line) > 1:
        ptt['tokens'][i].border = 1

for i in range(len(ptt['tokens'])):
    if any(True if t.border == 1 else False for t in ptt['tokens'][i:i+n-1]) and not ptt['tokens'][i].border:
        ptt['tokens'][i].border = 2
    if any(True if t.border == 1 else False for t in ptt['tokens'][max(0, i-n+2):i]) and not ptt['tokens'][i].border:
        ptt['tokens'][i].border = 2
    

In [ ]:
# DeepDocTection
ddt = {}

banned_tokens = {"©"}

text = pdf[0].text.replace("\n", " ").replace("- ", "")

tokens = [token for token in nlp.make_doc(text) if not token.is_punct and token.text not in banned_tokens]

ddt['tokens'] = tokens
ddt['ngrams'] = {" ".join(t.text for t in tokens[i:i+n]) for i in range(len(tokens) - n + 1)}


In [ ]:
def zip_print(zip):
    for i,j in zip:
        print(f"{i.token.text:30s} {j}")

zip_print(zip(ptt['tokens'][:30], ddt['tokens'][1:31]))


Tweede                         Tweede
Kamer                          Kamer
Directoraat-Generaal           nota
Migratie                       Beslisnota
Directie                       Kamerbrief
Regie                          inzake
Migratieketen                  verdeelbesluiten
Team                           1
Spreidingswet                  Aanleiding
Turfmarkt                      Op
147                            31
2511                           oktober
DP                             2024
Den                            ontving
Haag                           u
Postbus                        de
20011                          verslagen
2500                           van
EH                             de
Den                            commissarissen
Haag                           van
www.rijksoverheid.nl/jenv      de
Beslisnota                     Koning
Kamerbrief                     in
inzake                         het
verdeelbesluiten               kader
Datum                     

In [316]:
missing = [ngram for ngram in list(ptt['ngrams'].keys()) if not ngram in ddt['ngrams']]

for ngram in missing:
    for token in ptt['ngrams'][ngram]:
        token.missing += 1
        print(token)


Tweede = 'B'2(0:0) - missing 1 times.
Kamer = 'B'1(0:1) - missing 1 times.
Directoraat-Generaal = 'B'1(2:0) - missing 1 times.
Migratie = 'B'2(3:0) - missing 1 times.
Kamer = 'B'1(0:1) - missing 2 times.
Directoraat-Generaal = 'B'1(2:0) - missing 2 times.
Migratie = 'B'2(3:0) - missing 2 times.
Directie = 'B'2(4:0) - missing 1 times.
Den = 'B'2(9:2) - missing 1 times.
Haag = 'B'2(9:3) - missing 1 times.
www.rijksoverheid.nl/jenv = 'B'1(10:0) - missing 1 times.
Beslisnota = 'B'1(12:0) - missing 1 times.
Haag = 'B'2(9:3) - missing 2 times.
www.rijksoverheid.nl/jenv = 'B'1(10:0) - missing 2 times.
Beslisnota = 'B'1(12:0) - missing 2 times.
Kamerbrief = 'B'2(12:1) - missing 1 times.
www.rijksoverheid.nl/jenv = 'B'1(10:0) - missing 3 times.
Beslisnota = 'B'1(12:0) - missing 3 times.
Kamerbrief = 'B'2(12:1) - missing 2 times.
inzake = 'B'2(12:2) - missing 1 times.
Kamerbrief = 'B'2(12:1) - missing 3 times.
inzake = 'B'2(12:2) - missing 2 times.
verdeelbesluiten = 'B'1(12:3) - missing 1 times

In [318]:
html = BeautifulSoup(f'<html></html>', 'html.parser')

head = html.new_tag("head")
title = html.new_tag("title")
title.string = f"Evaluation of {FILE}"
head.append(title)
html.html.append(head)
    

In [319]:
line = 0

body = html.new_tag("body")
p = html.new_tag("p")

colors = ["white","rgba(255, 255, 0, 0.5)","rgba(255, 166, 0, 0.5)","rgba(255, 0, 0, 0.5)","rgba(139, 0, 0, 0.5)"]

for token in ptt['tokens']:
    if token.line != line:
        for i in range(token.line - line):
            p.append(html.new_tag("br"))
        line = token.line

    span = html.new_tag("span", style=f"background-color: {colors[token.missing]}; opacity: {'0.25' if token.border else '1'}")
    span.string = token.text
    p.append(span)

body.append(p)
html.html.append(body)



In [320]:
with open(f"{FILE+'.html'}", "w", encoding="utf-8") as file:
    file.write(html.prettify())


In [ ]:
MISTAKES = len([t for t in ptt['tokens'] if t.missing])
MISTAKES_BORDER = len([t for t in ptt['tokens'] if t.missing and t.border])

print(f"{MISTAKES} mistakes ({MISTAKES_BORDER}/{MISTAKES}): {((MISTAKES - MISTAKES_BORDER) / MISTAKES * 100):0.1f}%")


27 mistakes (27/27): 0.0%
